In [11]:
# Cell 1: Import Libraries and Setup Paths
import cudf
import cupy as cp
from cuml import RandomForestClassifier
from cuml.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from cuml.metrics.confusion_matrix import confusion_matrix
from sklearn.metrics import classification_report

# Define paths
processed_train_path = '/root/autodl-tmp/projects/SL_NSL/dataset/processed/bin/KDDTrain_processed.csv'
processed_test_path = '/root/autodl-tmp/projects/SL_NSL/dataset/processed/bin/KDDTest_processed.csv'

In [12]:
# Cell 2: Load training data
print("Loading processed training data...")
df_train = cudf.read_csv(processed_train_path)

# Select features and labels
X = df_train.drop(columns=['binary_label']).astype('float32')
y_binary = df_train['binary_label']

# Display data info
print("Feature matrix shape:", X.shape)
print("Label distribution:")
print(y_binary.value_counts().to_pandas())

Loading processed training data...
Feature matrix shape: (125973, 108)
Label distribution:
0    67343
1    58630
Name: binary_label, dtype: int64


In [13]:
# Cell 3: Split into training and validation sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y_binary, test_size=0.2, random_state=42,
)

# Data type conversion
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

print("Dataset split complete")
print(f"Training samples: {X_train.shape[0]}")
print(f"Validation samples: {X_test.shape[0]}")

Dataset split complete
Training samples: 100778
Validation samples: 25195


In [14]:
# Cell 4: Create and train Random Forest model
print("\nTraining Random Forest model on GPU...")
rf_binary = RandomForestClassifier(n_estimators=50, random_state=42, n_streams=1)
rf_binary.fit(X_train, y_train)

# Predict and evaluate on validation set
print("\nEvaluating on validation set...")
y_pred = rf_binary.predict(X_test)

# Data type conversion
y_pred = y_pred.astype('int32')
y_test = y_test.astype('int32')

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

# Convert to CPU for classification report
y_test_cpu = y_test.to_pandas()
y_pred_cpu = y_pred.to_pandas()
class_report = classification_report(y_test_cpu, y_pred_cpu)

print(f"Validation Accuracy: {accuracy:.2f}")
print("\nConfusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(class_report)


Training Random Forest model on GPU...

Evaluating on validation set...
Validation Accuracy: 1.00

Confusion Matrix:
[[13409    13]
 [   73 11700]]

Classification Report:
              precision    recall  f1-score   support

           0       0.99      1.00      1.00     13422
           1       1.00      0.99      1.00     11773

    accuracy                           1.00     25195
   macro avg       1.00      1.00      1.00     25195
weighted avg       1.00      1.00      1.00     25195



In [15]:
# Cell 5: Predict and evaluate on validation set
print("\nEvaluating on validation set...")

# Ensure y_test is int32 before prediction
y_test = y_test.astype('int32')

# Make predictions and convert to int32
y_pred = rf_binary.predict(X_test)
y_pred = cudf.Series(y_pred).astype('int32')

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

# Convert to CPU for classification report
y_test_cpu = y_test.to_pandas()
y_pred_cpu = y_pred.to_pandas()
class_report = classification_report(y_test_cpu, y_pred_cpu)

print(f"Validation Accuracy: {accuracy:.2f}")
print("\nConfusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(class_report)


Evaluating on validation set...
Validation Accuracy: 1.00

Confusion Matrix:
[[13409    13]
 [   73 11700]]

Classification Report:
              precision    recall  f1-score   support

           0       0.99      1.00      1.00     13422
           1       1.00      0.99      1.00     11773

    accuracy                           1.00     25195
   macro avg       1.00      1.00      1.00     25195
weighted avg       1.00      1.00      1.00     25195



In [16]:
# Cell 6: Load and evaluate on external test set
print("\nEvaluating on external test set...")
df_test = cudf.read_csv(processed_test_path)

X_external_test = df_test.drop(columns=['binary_label']).astype('float32')
y_external_test = df_test['binary_label'].astype('int32') 

# Make predictions and convert to int32
y_external_pred = rf_binary.predict(X_external_test)
y_external_pred = cudf.Series(y_external_pred).astype('int32')

# Calculate metrics
accuracy_external = accuracy_score(y_external_test, y_external_pred)
conf_matrix_external = confusion_matrix(y_external_test, y_external_pred)

# Convert to CPU for classification report
y_external_test_cpu = y_external_test.to_pandas()
y_external_pred_cpu = y_external_pred.to_pandas()
class_report_external = classification_report(y_external_test_cpu, y_external_pred_cpu)

print(f"External Test Set Accuracy: {accuracy_external:.2f}")
print("\nConfusion Matrix:")
print(conf_matrix_external)
print("\nClassification Report:")
print(class_report_external)


Evaluating on external test set...
External Test Set Accuracy: 0.81

Confusion Matrix:
[[8369 1342]
 [2916 9917]]

Classification Report:
              precision    recall  f1-score   support

           0       0.74      0.86      0.80      9711
           1       0.88      0.77      0.82     12833

    accuracy                           0.81     22544
   macro avg       0.81      0.82      0.81     22544
weighted avg       0.82      0.81      0.81     22544

